In [1]:
import glob
import math
import os
import shutil
import tempfile
import time

from torch.optim import Adam, SGD
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import (
    CacheDataset,
    DataLoader,
    ThreadDataLoader,
    Dataset,
    decollate_batch,
    set_track_meta,
)
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss, DiceCELoss
from monai.metrics import DiceMetric
from monai.networks.layers import Act, Norm
from monai.networks.nets import UNet, SwinUNETR, AHNet, VNet

from monai.transforms import (
    EnsureChannelFirstd,
    AsDiscrete,
    Compose,
    CropForegroundd,
    EnsureTyped,
    FgBgToIndicesd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
)
from monai.utils import set_determinism


from monai.data import (
    ThreadDataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    set_track_meta,
)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"  # Use the 3rd and 4th GPU. Indexing starts from 0.

# for profiling
import nvtx
from monai.utils.nvtx import Range
import contextlib  # to improve code readability (combining training/validation loop with and without profiling)

#print_config()
set_determinism(0)

2023-10-09 04:26:21.375585: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
In the future `np.object` will be defined as the corresponding NumPy scalar.
In the future `np.object` will be defined as the corresponding NumPy scalar.


In [2]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(f"root dir is: {root_dir}")

current_directory = os.getcwd()

out_dir = os.path.join(current_directory,"outputs/")

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

root dir is: /tmp/tmp5y4a70vo


In [3]:
import os
import sys
import gc
import ast
import cv2
import time
import timm
import pickle
import random
import pydicom
import argparse
import warnings
import numpy as np
import pandas as pd
from glob import glob
import nibabel as nib
from PIL import Image
from tqdm import tqdm
import albumentations
from pylab import rcParams
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from sklearn.model_selection import KFold, StratifiedKFold
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple, Optional, List
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
import torch.nn.functional as F


from timm.layers.conv2d_same import Conv2dSame

def convert_3d(module):

    module_output = module
    if isinstance(module, torch.nn.BatchNorm2d):
        module_output = torch.nn.BatchNorm3d(
            module.num_features,
            module.eps,
            module.momentum,
            module.affine,
            module.track_running_stats,
        )
        if module.affine:
            with torch.no_grad():
                module_output.weight = module.weight
                module_output.bias = module.bias
        module_output.running_mean = module.running_mean
        module_output.running_var = module.running_var
        module_output.num_batches_tracked = module.num_batches_tracked
        if hasattr(module, "qconfig"):
            module_output.qconfig = module.qconfig
            
    elif isinstance(module, Conv2dSame):
        module_output = Conv3dSame(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
        )
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.Conv2d):
        module_output = torch.nn.Conv3d(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
            padding_mode=module.padding_mode
        )
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.MaxPool2d):
        module_output = torch.nn.MaxPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            dilation=module.dilation,
            ceil_mode=module.ceil_mode,
        )
    elif isinstance(module, torch.nn.AvgPool2d):
        module_output = torch.nn.AvgPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            ceil_mode=module.ceil_mode,
        )

    for name, child in module.named_children():
        module_output.add_module(
            name, convert_3d(child)
        )
    del module

    return module_output

# Calculate symmetric padding for a convolution
def get_padding(kernel_size: int, stride: int = 1, dilation: int = 1, **_) -> int:
    padding = ((stride - 1) + dilation * (kernel_size - 1)) // 2
    return padding


# Calculate asymmetric TensorFlow-like 'SAME' padding for a convolution
def get_same_padding(x: int, k: int, s: int, d: int):
    return max((math.ceil(x / s) - 1) * s + (k - 1) * d + 1 - x, 0)


# Can SAME padding for given args be done statically?
def is_static_pad(kernel_size: int, stride: int = 1, dilation: int = 1, **_):
    return stride == 1 and (dilation * (kernel_size - 1)) % 2 == 0


# Dynamically pad input x with 'SAME' padding for conv with specified args
def pad_same(x, k: List[int], s: List[int], d: List[int] = (1, 1, 1), value: float = 0):
    ih, iw, iz = x.size()[-3:]
    pad_h = get_same_padding(ih, k[0], s[0], d[0])
    pad_w = get_same_padding(iw, k[1], s[1], d[1])
    pad_z = get_same_padding(iz, k[2], s[2], d[2])
    if pad_h > 0 or pad_w > 0 or pad_z > 0:
        x = F.pad(x, [pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2, pad_z // 2, pad_z - pad_z // 2], value=value)
    return x


def get_padding_value(padding, kernel_size, **kwargs) -> Tuple[Tuple, bool]:
    dynamic = False
    if isinstance(padding, str):
        # for any string padding, the padding will be calculated for you, one of three ways
        padding = padding.lower()
        if padding == 'same':
            # TF compatible 'SAME' padding, has a performance and GPU memory allocation impact
            if is_static_pad(kernel_size, **kwargs):
                # static case, no extra overhead
                padding = get_padding(kernel_size, **kwargs)
            else:
                # dynamic 'SAME' padding, has runtime/GPU memory overhead
                padding = 0
                dynamic = True
        elif padding == 'valid':
            # 'VALID' padding, same as padding=0
            padding = 0
        else:
            # Default to PyTorch style 'same'-ish symmetric padding
            padding = get_padding(kernel_size, **kwargs)
    return padding, dynamic


def conv3d_same(
        x, weight: torch.Tensor, bias: Optional[torch.Tensor] = None, stride: Tuple[int, int, int] = (1, 1, 1),
        padding: Tuple[int, int, int] = (0, 0, 0), dilation: Tuple[int, int, int] = (1, 1, 1), groups: int = 1):
    x = pad_same(x, weight.shape[-3:], stride, dilation)
    return F.conv3d(x, weight, bias, stride, (0, 0, 0), dilation, groups)


class Conv3dSame(nn.Conv3d):
    """ Tensorflow like 'SAME' convolution wrapper for 3d convolutions
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        super(Conv3dSame, self).__init__(
            in_channels, out_channels, kernel_size, stride, 0, dilation, groups, bias)

    def forward(self, x):
        return conv3d_same(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)


def create_conv3d_pad(in_chs, out_chs, kernel_size, **kwargs):
    padding = kwargs.pop('padding', '')
    kwargs.setdefault('bias', False)
    padding, is_dynamic = get_padding_value(padding, kernel_size, **kwargs)
    if is_dynamic:
        return Conv3dSame(in_chs, out_chs, kernel_size, **kwargs)
    else:
        return nn.Conv3d(in_chs, out_chs, kernel_size, padding=padding, **kwargs)

kernel_type = 'timm3d_res18d_unet4b_128_128_128_dsv2_flip12_shift333p7_gd1p5_bs4_lr3e4_20x50ep'
load_kernel = None
load_last = True
n_blocks = 4
n_folds = 5
backbone = 'resnet18d'

init_lr = 3e-3
batch_size = 4
drop_rate = 0.
drop_path_rate = 0.
loss_weights = [1, 1]
p_mixup = 0.1
drop_rate = 0.
drop_path_rate = 0.
out_dim = 6
        
import torch
import torch.nn as nn
import timm
import segmentation_models_pytorch as smp

class TimmSegModel(nn.Module):
    def __init__(self, backbone, segtype='unet', pretrained=False, num_classes=6):
        super(TimmSegModel, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            in_chans=3,
            features_only=True,
            drop_rate=0.1,
            drop_path_rate=0.1,
            pretrained=pretrained
        )

        # Modify the first layer of the encoder for 1-channel input
        self.encoder.conv1 = nn.Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)

        self.decoder = smp.Unet(
            encoder_name=backbone,
            encoder_depth=5,
            encoder_weights=None,  # Weights are already loaded in the encoder part
            decoder_use_batchnorm=True,
            decoder_channels=[256, 128, 64, 32, 16],
            decoder_attention_type='scse',
            in_channels=1,
            classes=num_classes,
            activation=None,  # Use 'None' because we handle activation in the loss function (e.g., CrossEntropy)
        )

    def forward(self, x):
        return self.decoder(x)

In [4]:
# Redefining the SEGMENTATION_CODES dictionary and mock data
SEGMENTATION_CODES = {
    #0: 'background',
    1: 'liver',
    2: 'spleen',
    3: 'kidney_left',
    4: 'kidney_right',
    5: 'bowel',
}

In [5]:
def mark_organ_presence_matrix(depth, bounding_boxes):
    organ_order = ['liver', 'kidneys', 'spleen', 'bowel']
    
    organ_presence_matrix = np.zeros((depth, len(organ_order)), dtype=int)
    
    for idx, organ_name in enumerate(organ_order):
        if organ_name in bounding_boxes:
            start, end = bounding_boxes[organ_name]["depth_range"]
            organ_presence_matrix[start:end+1, idx] = 1

    return organ_presence_matrix

def fourier_smoothing(y, cutoff_fraction=0.1):
    # Compute the FFT of the input data
    y_fft = np.fft.fft(y)
    
    # Zero out the higher frequencies
    cutoff_idx = int(cutoff_fraction * len(y_fft))
    y_fft[cutoff_idx:-cutoff_idx] = 0
    
    # Compute the inverse FFT to get the smoothed data
    y_smoothed = np.fft.ifft(y_fft).real  # Only take the real part
    
    return y_smoothed
    

# Function to save numpy array as .nii.gz
def save_nii(data, filename):
    img = nib.Nifti1Image(data, affine=np.eye(4))
    nib.save(img, filename)

In [46]:
def extract_organ_subregion(volume, bounding_boxes, organ_name):
    # Ensure the provided organ_name exists in the bounding_boxes
    if organ_name not in bounding_boxes:
        raise ValueError(f"No bounding box found for organ: {organ_name}")
    
    # Extract the coordinates from the specified organ's bounding box
    top_left_front = bounding_boxes[organ_name]["top_left_front"]
    bottom_right_back = bounding_boxes[organ_name]["bottom_right_back"]

    
    subregion = volume[top_left_front[0]:bottom_right_back[0]+1, :, :]

    
    return subregion


def extract_images_from_volume(volume, device, output_directory, local_bbox, resize_to=(100, 224, 224), use_three_channel=True, overlap=True):
    """
    Given a 3D volume, this function will preprocess each slice, round the pixel values, and save them using the local bounding boxes.
    """
    # Convert ndarray to tensor
    if isinstance(volume, np.ndarray):
        volume = torch.from_numpy(volume.astype(np.float32)).to(device)

    # Resize the slices
    D, H, W = volume.shape  # Assuming volume has shape (D, H, W)
    desired_D = min(D, 300)
    volume = torch.nn.functional.interpolate(volume.unsqueeze(0).unsqueeze(0), size=(desired_D, H, W), mode='trilinear', align_corners=True).squeeze(0).squeeze(0)

    # Define the step based on whether overlap is desired
    step = 1 if overlap else 3

    # Iterate through the volume
    depth = volume.shape[0]

    for idx in range(1, depth-1, step):  # Avoiding starting at index 0 and ending at index -1
        if use_three_channel:
            # Create 3-channel image
            slices = volume[max(idx-1, 0):min(idx+2, depth), :, :]
            
            if output_directory:
                # Convert tensor slices to numpy and then to a PIL Image
                # Stack slices along channel dimension
                slices = slices.byte()
                rgb_slices = torch.stack([slices[i] for i in range(slices.shape[0])], dim=0)
                
                # Extract region of interest using local_bbox
                x, y, w, h = local_bbox
                rgb_slices_cropped = rgb_slices[:, y:y+h, x:x+w]
                
                # Convert cropped tensor to PIL Image
                rgb_img = Image.fromarray(rgb_slices_cropped.permute(1, 2, 0).cpu().numpy())
                
                # Save the RGB image
                rgb_img.save(os.path.join(output_directory, f"{idx}.jpg"))

                del rgb_img
    del volume
    # Free up CUDA memory
    torch.cuda.empty_cache()

    print(f"Extracted images saved to {output_directory}")


In [8]:
import os
import pandas as pd
from tqdm import tqdm

# Get the current working directory
current_directory = os.getcwd()

# Construct the path to the train_images directory located one level up
train_base_directory = os.path.join(current_directory, 'train_images')

# Collect all the DICOM file paths along with patient IDs and series ID
dicom_data = []
for patient_id in tqdm(os.listdir(train_base_directory), desc="Processing Patients"):
    patient_dir = os.path.join(train_base_directory, patient_id)
    for series_id in os.listdir(patient_dir):
        series_dir = os.path.join(patient_dir, series_id)

        # Check if the series directory is empty
        if not os.listdir(series_dir):
            print(f"{series_dir} is empty. Skipping...")
            continue

        # Check if all files in the series directory are DICOM files
        all_files_are_dicom = all([f.endswith('.dcm') for f in os.listdir(series_dir)])
        if not all_files_are_dicom:
            print(f"Non-DICOM files found in {series_dir}. Skipping...")
            continue

        dicom_data.append({'series_dir': series_dir, 'patient_id': patient_id, 'series_id': series_id})

# Create a DataFrame using the collected data
dicom_df = pd.DataFrame(dicom_data)

dicom_df['patient_duplicate_count'] = dicom_df.groupby('patient_id')['patient_id'].transform('count')

# Transformation pipeline
test_org_transforms = Compose(
    [
        LoadImaged(keys="image"),
        EnsureChannelFirstd(keys="image"),
        Orientationd(keys=["image"], axcodes="RAS"),
        Spacingd(keys=["image"], pixdim=(2.0, 2.0, 2.0), mode="bilinear"),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
    ]
)

Processing Patients: 100%|████████████████████████████████████████████████████████████████████████████| 3147/3147 [00:04<00:00, 687.61it/s]
monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


In [45]:
import psutil
# Get the virtual memory status
memory_info = psutil.virtual_memory()

# Print the free memory in bytes
print(f"Free RAM: {memory_info.free} bytes")

# If you want the free RAM in GB
print(f"Free RAM: {memory_info.free / (1024 ** 3)} GB")


Free RAM: 33134325760 bytes
Free RAM: 30.858745574951172 GB


In [55]:
import os
import pydicom as dicom
import numpy as np
from skimage.transform import resize
from scipy.ndimage import zoom
from numba import jit

@jit(nopython=True)
def adjust_pixel_values(pixel_array, slope, intercept, center, width):
    pixel_array = (pixel_array * slope) + intercept
    low = center - width / 2
    high = center + width / 2
    pixel_array = np.clip(pixel_array, low, high)
    pixel_array -= np.min(pixel_array)
    pixel_array = (pixel_array / np.max(pixel_array) * 255).astype(np.int16)
    return pixel_array


def sample_patient_volume(folder, depth_downsample_rate=None, lw_downsample_rate=None, 
                          adjust_pixel_spacing="no", standardize_pixel_array="yes", 
                          target_pixel_spacing=[1.0, 1.0], target_thickness=None):
    """
    Load and optionally adjust and standardize the pixel array from DICOM metadata.
    """
    
    filenames = sorted([int(filename.split('.')[0]) for filename in os.listdir(folder)])
    filenames = [str(filename) + '.dcm' for filename in filenames]
    filenames_len = len(filenames)
    first_part = int(filenames_len*0.35)
    last_part = int(filenames_len*0.65)
    
    volume = []
    original_thicknesses = []

    bbox_list = []
    counter = 0
    
    for filename in filenames:
        filepath = os.path.join(folder, filename)
        dcm = dicom.dcmread(filepath)
        original_thicknesses.append(float(dcm.SliceThickness))
        
        pixel_array = dcm.pixel_array

        # Adjust pixel spacing if required
        if standardize_pixel_array.lower() == "yes":
            if dcm.PixelRepresentation == 1:
                bit_shift = dcm.BitsAllocated - dcm.BitsStored
                dtype = pixel_array.dtype 
                pixel_array = (pixel_array << bit_shift).astype(dtype) >> bit_shift

            intercept = float(dcm.RescaleIntercept)
            slope = float(dcm.RescaleSlope)
            center = int(dcm.WindowCenter)
            width = int(dcm.WindowWidth)
            pixel_array = adjust_pixel_values(pixel_array, slope, intercept, center, width)
            
        if adjust_pixel_spacing.lower() == "yes":
            row_spacing, col_spacing = dcm.PixelSpacing
            new_shape = (int(pixel_array.shape[0] * (row_spacing/target_pixel_spacing[0])), 
                         int(pixel_array.shape[1] * (col_spacing/target_pixel_spacing[1])))
            pixel_array = resize(pixel_array, new_shape)

        if lw_downsample_rate:
            pixel_array = pixel_array[::lw_downsample_rate, ::lw_downsample_rate]

        if counter > first_part and counter < last_part:
            
            bbox = get_bounding_box(pixel_array)
            
            bbox_list.append(bbox)

        volume.append(pixel_array)

        counter = counter + 1

        
    volume = np.array(volume)
    print(volume.shape)
    # Adjust the slice thickness if needed
    if target_thickness is not None:
        original_thickness = np.mean(original_thicknesses)
        z_zoom_factor = original_thickness / target_thickness
        volume = zoom(volume, (z_zoom_factor, 1, 1), order=1)  # order=1 for bilinear interpolation

    bbox_array = np.array(bbox_list)

    #areas = bbox_array[:, 2] * bbox_array[:, 3]  # width * height for each bbox
    areas = [box[2] * box[3] for box in bbox_list]  # width * height for each bbox

    # Get the index of the maximum area
    max_area_index = np.argmax(areas)
    
    return volume, bbox_list[max_area_index]

def get_bounding_box(image):
    # Normalize the image to the range [0, 255]
    normalized_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Now, apply Otsu's thresholding
    _, binary_image = cv2.threshold(normalized_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Noise reduction using morphological opening
    kernel = np.ones((5,5), np.uint8)
    opened_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)
    opened_image = cv2.morphologyEx(opened_image, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(opened_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assuming the largest contour corresponds to the abdominal region, find its bounding box
    if contours:
        # Sort contours by area and take the largest one
        largest_contour = max(contours, key=cv2.contourArea)

        # Get bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

    else:
        x, y, w, h = 0, 0, 0, 0  # Default values in case no contours are found

    # The bounding box is represented by the top-left corner (x, y) and the width and height (w, h)
    bounding_box = (x, y, w, h)
    
    return bounding_box

In [11]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

# Load data
train_info = pd.read_csv('train.csv')
train_info.patient_id = train_info.patient_id.astype(str)

curr_df = dicom_df#dicom_df[dicom_df.patient_duplicate_count == 2]
train_info = train_info[train_info.patient_id.isin(curr_df.patient_id)]
#train_info = train_info[train_info.extravasation_injury == 1]
curr_df = curr_df[curr_df.patient_id.isin(train_info.patient_id)]

unique_patients = np.unique(curr_df.patient_id)

# Directory where the .nii.gz files are saved
vol_directory = "/workspace/0728tot/ATD/patient_nii"

seg_model_device = torch.device("cuda:0")
feature_extractor_device = torch.device("cuda:1")

# Create the model
seg_model = TimmSegModel(backbone = 'resnet18', segtype='unet', pretrained = True)
seg_model = convert_3d(seg_model)
# Test the model (just to ensure everything is working)
seg_model = seg_model.to(seg_model_device)

seg_model.load_state_dict(torch.load(os.path.join(current_directory, "resnet18_unet_best_metric_model.pt")))
seg_model.eval()

TimmSegModel(
  (encoder): FeatureListNet(
    (conv1): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_s

In [12]:
mapping = train_info[['patient_id','any_injury']].sort_values(by='any_injury', ascending=False)
unique_patients = np.array(mapping.patient_id)

In [13]:
def process_and_get_bounding_boxes_v6(tensor):
    """
    Compute bounding boxes, determine the voxel count per depth, and the inner region for each organ.
    Also computes a bounding box that encompasses all organs based on the inner_slices.
    
    Args:
    - tensor (numpy.ndarray): The input tensor with shape (D, H, W).
    
    Returns:
    - dict: Bounding boxes, voxel counts per depth, and inner regions for each organ.
            Also includes the encompassing bounding box for all organs.
    """
    
    # Initialize bounding boxes dictionary
    bounding_boxes = {}
    
    # List to store all coordinates within the inner_slices for all organs
    all_coords_in_inner_slices = []
    
    for organ_label, organ_name in SEGMENTATION_CODES.items():
        if organ_label == 0:  # Skip background
            continue

        # Fourier-based smoothing logic based on organ_name
        fourier_factor = 0
        threshold_factor = 0
        if organ_name == 'liver':
            fourier_factor = 0.05
            threshold_factor = 0.2
        elif organ_name == 'spleen':
            fourier_factor = 0.05
            threshold_factor = 0.3
        elif organ_name == 'kidney_left':
            fourier_factor = 0.05
            threshold_factor = 0.3
        elif organ_name == 'kidney_right':
            fourier_factor = 0.05
            threshold_factor = 0.35
        elif organ_name == 'bowel':
            fourier_factor = 0.025
            threshold_factor = 0.2
        
        # Find the slices where the organ appears
        slices = np.where(np.any(tensor[0] == organ_label, axis=(1, 2)))[0]
        
        if len(slices) == 0:  # If organ does not appear, continue to next organ
            continue
        
        # Compute voxel count for each slice and store in a dictionary with slice index as key
        voxel_counts_per_depth = {slice_idx: np.sum(tensor[0, slice_idx] == organ_label) for slice_idx in slices}

        # Fourier-based smoothing
        depths = list(voxel_counts_per_depth.keys())
        values = list(voxel_counts_per_depth.values())
        smoothed_values = fourier_smoothing(values, fourier_factor)
        voxel_counts_smoothed = dict(zip(depths, smoothed_values))
        
        # Thresholding logic
        max_value = np.max(smoothed_values)
        threshold = threshold_factor * max_value
        in_threshold = [depth for depth, value in voxel_counts_smoothed.items() if value >= threshold]

        # Gather all coordinates for this organ that are within the inner_slices
        # coords = np.argwhere((tensor[0] == organ_label) & np.isin(tensor[0], in_threshold))
        depth_mask = np.isin(np.arange(tensor.shape[1]), in_threshold)
        organ_mask = tensor[0] == organ_label
        combined_mask = np.logical_and(organ_mask, depth_mask[:, None, None])

        coords = np.argwhere(combined_mask)

        all_coords_in_inner_slices.extend(coords)
        
        # Bounding box computation for this organ
        min_coords = coords.min(axis=0)
        max_coords = coords.max(axis=0)
        
        bounding_boxes[organ_name] = {
            "top_left_front": tuple(min_coords),
            "bottom_right_back": tuple(max_coords),
            "depth_range": (min_coords[0], max_coords[0]),
            "voxel_counts_per_depth": voxel_counts_per_depth,
            "inner_slices": in_threshold
        }
    
    # Compute the encompassing bounding box for all organs based on inner_slices
    all_coords_array = np.array(all_coords_in_inner_slices)

    min_encompassing_coords = all_coords_array.min(axis=0)
    max_encompassing_coords = all_coords_array.max(axis=0)
    bounding_boxes["encompassing"] = {
        "top_left_front": tuple(min_encompassing_coords),
        "bottom_right_back": tuple(max_encompassing_coords),
        "depth_range": (min_encompassing_coords[0], max_encompassing_coords[0])
    }
        
    if "kidney_left" in bounding_boxes and "kidney_right" in bounding_boxes:
        kidney_left_bbox = bounding_boxes["kidney_left"]
        kidney_right_bbox = bounding_boxes["kidney_right"]
        
        merged_top_left_front = tuple(np.minimum(kidney_left_bbox["top_left_front"], kidney_right_bbox["top_left_front"]))
        merged_bottom_right_back = tuple(np.maximum(kidney_left_bbox["bottom_right_back"], kidney_right_bbox["bottom_right_back"]))
        
        bounding_boxes["kidneys"] = {
            "top_left_front": merged_top_left_front,
            "bottom_right_back": merged_bottom_right_back,
            "depth_range": (merged_top_left_front[0], merged_bottom_right_back[0])
        }
    elif "kidney_left" in bounding_boxes:
        bounding_boxes["kidneys"] = bounding_boxes["kidney_left"]
    elif "kidney_right" in bounding_boxes:
        bounding_boxes["kidneys"] = bounding_boxes["kidney_right"]
        
    return bounding_boxes

In [57]:
# Iterate over unique patients
save_images = True
for patient_id in tqdm(unique_patients):
    eval_df = curr_df[curr_df.patient_id == patient_id]
    paths = np.array(eval_df.series_dir)
    series_names = np.array(eval_df.series_id)
    
    volumes = []
    
    # For each series in a patient, sample the volume
    for path, series_name in zip(paths, series_names):
        vol, bbox_list = sample_patient_volume(path, depth_downsample_rate=None, lw_downsample_rate=None, 
                                  adjust_pixel_spacing="no", standardize_pixel_array="yes", 
                                  target_pixel_spacing=[1.0, 1.0], target_thickness=None)
        volumes.append((series_name, vol, bbox_list))
    
    # Filtering based on depth criterion
    depths = [vol.shape[0] for _, vol, _ in volumes]
    max_depth = max(depths)
    
    # Remove volumes that don't meet the depth criterion from the list
    volumes = [(series_name, vol, bbox) for series_name, vol, bbox in volumes if vol.shape[0] >= 0.4 * max_depth]
    
    # Save volumes and keep track of saved filepaths
    saved_filepaths = []
    
    for series_name, vol, bbox in volumes:
        filename = os.path.join(vol_directory, f"{series_name}.nii.gz")
        save_nii(vol, filename)
        saved_filepaths.append(filename)
    
    # Create a data list for the Dataset using the saved_filepaths list
    data_list = [{"image": filepath} for filepath in saved_filepaths]
    
    # Create DataLoader
    test_org_ds = Dataset(data=data_list, transform=test_org_transforms)
    test_org_loader = DataLoader(test_org_ds, batch_size=1, num_workers=0)
    
    # Initialize an empty list to store bounding box data
    bbox_data_list = []

    with torch.no_grad():
        # Loop through both filepaths (from DataLoader) and volumes
        for (test_data, (series_name, vol, bbox)) in zip(test_org_loader, volumes):
    
            test_inputs = test_data["image"].to(seg_model_device)
            roi_size = (96, 96, 96)
            sw_batch_size = 4
            curr_data = sliding_window_inference(test_inputs, roi_size, sw_batch_size, seg_model)
            ideal_size = list(test_data['image_meta_dict']['spatial_shape'][0].cpu().detach().numpy())
            # Process the data
            segmented = curr_data.cpu()
            segmented_shape = segmented.shape
                
            merged = torch.argmax(segmented, dim=1)# + 1  # add 1 to move the range from 0-5 to 1-6
            
            bounding_boxes_result_v3 = process_and_get_bounding_boxes_v6(merged)

            # Given scale factors
            depth_scale_factor = ideal_size[0] / segmented_shape[2]
            height_scale_factor = ideal_size[1] / segmented_shape[3]
            width_scale_factor = ideal_size[2] / segmented_shape[4]
            
            # Scale the bounding box coordinates
            for organ_name, box_info in bounding_boxes_result_v3.items():
                # Scaling top_left_front
                min_coords = box_info["top_left_front"]
                scaled_min_coords = (
                    round(min_coords[0] * depth_scale_factor),
                    round(min_coords[1] * height_scale_factor),
                    round(min_coords[2] * width_scale_factor)
                )
                box_info["top_left_front"] = scaled_min_coords
                
                # Scaling bottom_right_back
                max_coords = box_info["bottom_right_back"]
                scaled_max_coords = (
                    round(max_coords[0] * depth_scale_factor),
                    round(max_coords[1] * height_scale_factor),
                    round(max_coords[2] * width_scale_factor)
                )
                box_info["bottom_right_back"] = scaled_max_coords
                
                # Scaling depth_range
                depth_range = box_info["depth_range"]
                scaled_depth_range = (
                    round(depth_range[0] * depth_scale_factor),
                    round(depth_range[1] * depth_scale_factor)
                )
                box_info["depth_range"] = scaled_depth_range
    
            
            del merged
            del segmented
            del curr_data
            del ideal_size

            if save_images==True:
                organ_presence_matrix = mark_organ_presence_matrix(vol.shape[0], bounding_boxes_result_v3)
                base_directory_2 = os.path.join(current_directory, "volume_info")
                
                if not os.path.exists(base_directory_2):
                    os.makedirs(base_directory_2)  # Create the 'entire_volume_features' directory
                
                feature_path = os.path.join(base_directory_2, f"{series_name}_slice_labels.npy")
                np.save(feature_path, organ_presence_matrix)
            
            torch.cuda.empty_cache()
                # Ensure each organ has a bounding box. If not, set its volume to None.

            organs_volumes = {
                "encompassing": extract_organ_subregion(vol, bounding_boxes_result_v3, 'encompassing') if "encompassing" in bounding_boxes_result_v3 else None,      
            }

            top_left_front = bounding_boxes_result_v3['encompassing']["top_left_front"]
            bottom_right_back = bounding_boxes_result_v3['encompassing']["bottom_right_back"]


            local_bbox = bbox
            

            for organ_name, organ_vol in organs_volumes.items():
                # Only proceed if the organ has a valid volume (not None)
                patient_directory_1 = None
                
                if organ_vol is not None and organ_vol.shape[1]>100 and organ_vol.shape[2]>100:
                    
                    if save_images==True and organ_name == 'encompassing':
                        base_directory_1 = os.path.join(current_directory, "volume_images")
                        if not os.path.exists(base_directory_1):
                            os.makedirs(base_directory_1)  # Create the 'entire_volume_features' directory
                    
                        patient_directory_1 = os.path.join(base_directory_1, str(series_name))
                        if not os.path.exists(patient_directory_1):
                            os.makedirs(patient_directory_1)  # Create the subdirectory for the 'series_name'
                            
                        extract_images_from_volume(organ_vol, 
                                                   feature_extractor_device, 
                                                   patient_directory_1,
                                                   local_bbox,
                                                   resize_to=(100, 512, 512), 
                                                   use_three_channel=True, 
                                                   overlap=False)
                    
            del organs_volumes
    del volumes, test_inputs

  0%|                                                                                                             | 0/3147 [00:00<?, ?it/s]

(1022, 512, 512)
(1044, 512, 512)
Extracted images saved to /workspace/0728tot/ATD/volume_images/21057


  0%|                                                                                                  | 1/3147 [00:57<50:19:12, 57.58s/it]

Extracted images saved to /workspace/0728tot/ATD/volume_images/51033
(685, 512, 512)
(679, 512, 512)
Extracted images saved to /workspace/0728tot/ATD/volume_images/10242


  0%|                                                                                                  | 2/3147 [01:35<40:04:43, 45.88s/it]

...

Extracted images saved to /workspace/0728tot/ATD/volume_images/30005
(188, 512, 512)


100%|██████████████████████████████████████████████████████████████████████████████████████████████▉| 3146/3147 [12:51:22<00:11, 11.92s/it]

Extracted images saved to /workspace/0728tot/ATD/volume_images/49265
(217, 512, 512)
(70, 512, 512)


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 3147/3147 [12:51:29<00:00, 14.71s/it]

Extracted images saved to /workspace/0728tot/ATD/volume_images/52826


In [33]:
dicom_df.to_csv('patient_series_mapping.csv')